# Lab: Bring your own custom container with Amazon SageMaker

## Overview
Here, we’ll show how to package a simple Python example which showcases the decision tree algorithm from the widely used scikit-learn machine learning package. The example is purposefully fairly trivial since the point is to show the surrounding structure that you’ll want to add to your own code so you can train and host it in Amazon SageMaker.

The ideas shown here will work in any language or environment. You’ll need to choose the right tools for your environment to serve HTTP requests for inference, but good HTTP environments are available in every language these days.

In this example, we use a single image to support training and hosting. This is easy because it means that we only need to manage one image and we can set it up to do everything. Sometimes you’ll want separate images for training and hosting because they have different requirements. Just separate the parts discussed below into separate Dockerfiles and build two images. Choosing whether to have a single image or two images is really a matter of which is more convenient for you to develop and manage.

If you’re only using Amazon SageMaker for training or hosting, but not both, there is no need to build the unused functionality into your container.



## Building the container
Docker provides a simple way to package arbitrary code into an image that is totally self-contained. Once you have an image, you can use Docker to run a container based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

### Parts of the container
In the container directory are all the components you need to package the sample algorithm for Amazon SageMager:

```
.
|-- Dockerfile
|-- build_and_push.sh
`-- decision_trees
    |-- nginx.conf
    |-- predictor.py
    |-- serve
    |-- train
    `-- wsgi.py
```

Let’s discuss each of these in turn:

- Dockerfile describes how to build your Docker container image. More details below:
- build_and_push.sh is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We’ll invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
- decision_trees is the directory which contains the files that will be installed in the container.
- local_test is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We’ll walk through local testing later in this notebook.

In this simple application, we only install five files in the container.

The files that we’ll put in the container are:

- nginx.conf is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
- predictor.py is the program that actually implements the Flask web server and the decision tree predictions for this app. You’ll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
- serve is the program started when the container is started for hosting. It simply launches the gunicorn server which runs multiple instances of the Flask app defined in predictor.py. You should be able to take this file as-is.
- train is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
- wsgi.py is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

In summary, the two files you will probably want to change for your application are train and predictor.py

### The Dockerfile
The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations.

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

## Building and registering the container

In [13]:
# cell 2
!pip install sagemaker-studio-image-build

You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


> *In the next cell, if you run into IAM permission issue related to CodeBuild, make sure that you follow the steps outlined in the immersion day lab instructions*

In [15]:
%%sh

# Cell #3

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

sm-docker build .  --repository sagemaker-decision-trees:latest

...[Container] 2021/08/12 03:36:38 Waiting for agent ping

[Container] 2021/08/12 03:36:41 Waiting for DOWNLOAD_SOURCE
[Container] 2021/08/12 03:36:41 Phase is DOWNLOAD_SOURCE
[Container] 2021/08/12 03:36:41 CODEBUILD_SRC_DIR=/codebuild/output/src437531969/src
[Container] 2021/08/12 03:36:41 YAML location is /codebuild/output/src437531969/src/buildspec.yml
[Container] 2021/08/12 03:36:41 Processing environment variables
[Container] 2021/08/12 03:36:41 No runtime version selected in buildspec.
[Container] 2021/08/12 03:36:41 Moving to directory /codebuild/output/src437531969/src
[Container] 2021/08/12 03:36:41 Registering with agent
[Container] 2021/08/12 03:36:41 Phases found in YAML: 3
[Container] 2021/08/12 03:36:41  POST_BUILD: 3 commands
[Container] 2021/08/12 03:36:41  PRE_BUILD: 9 commands
[Container] 2021/08/12 03:36:41  BUILD: 4 commands
[Container] 2021/08/12 03:36:41 Phase complete: DOWNLOAD_SOURCE State: SUCCEEDED
[Container] 2021/08/12 03:36:41 Phase context status code:  M

## Using the container
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [9]:
# Cell #4

# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

The session remembers our connection parameters to SageMaker. We’ll use it to perform all of our SageMaker operations.

In [10]:
# Cell #5

import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

When training large models with huge amounts of data, you’ll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we’re using some the classic Iris dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [6]:
# Cell #6

WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In order to use SageMaker to fit our algorithm, we’ll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above.

Then we use `fit()` on the estimator to train against the data that we uploaded above.

In [ ]:
# Cell #7

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, instance_count=1, instance_type='ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
file_location = data_location + '/iris.csv'
tree.fit(file_location)